In [1]:
# ------------------------------------------define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()

# this should import glob, os, and many other standard libs
from src.utils.Notebook_imports import *

# local imports
from src.utils.Utils_io import Console_and_file_logger, init_config
from src.visualization.Visualize import show_2D_or_3D

# import external libs
import cv2
import pandas as pd

EXPERIMENT = 'temp/MotionGenerator-tests'

# ------------------------------------------generator hyperparameters
SEED = 42 # define a seed for the generator shuffle
BATCHSIZE = 1 # 32, 64, 24, 16, 1 for 3D use: 4
GENERATOR_WORKER = BATCHSIZE # if not set, use batchsize
DIM = [60, 192, 192] # network input params for spacing of 3, (z,y,x)
SPACING = [2, 2, 2] # if resample, resample to this spacing, (z,y,x)
# maybe we can drop this channel?
#IMG_CHANNELS = 1 # Currently our model needs that image channel
MASK_VALUES = [1, 2, 3]  #channel order: Background, RV, MYO, LV
MASK_CLASSES = len(MASK_VALUES) # no of labels
MASKING_IMAGE = True
MASKING_VALUES = [2]
BORDER_MODE = cv2.BORDER_REFLECT_101 # border mode for the data generation
IMG_INTERPOLATION = cv2.INTER_LINEAR # image interpolation in the genarator
MSK_INTERPOLATION = cv2.INTER_NEAREST # mask interpolation in the generator
AUGMENT = True # a compose of 2D augmentation (grid distortion, 90degree rotation, brightness and shift)
AUGMENT_PROB = 1
SHUFFLE = True
RESAMPLE = True
SCALER = 'Standard' # MinMax Standard or Robust

Console_and_file_logger(EXPERIMENT, logging.INFO)
config = init_config(config=locals(), save=True)
print(config)

search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/dynamic-cmr-models


2021-02-08 19:37:08,628 INFO -------------------- Start --------------------
2021-02-08 19:37:08,629 INFO Working directory: /mnt/ssd/git/dynamic-cmr-models.
2021-02-08 19:37:08,629 INFO Log file: ./logs/temp/MotionGenerator-tests.log
2021-02-08 19:37:08,629 INFO Log level for console: INFO


{'SEED': 42, 'EXPERIMENT': 'temp/MotionGenerator-tests', 'BATCHSIZE': 1, 'GENERATOR_WORKER': 1, 'DIM': [60, 192, 192], 'SPACING': [2, 2, 2], 'MASK_VALUES': [1, 2, 3], 'MASK_CLASSES': 3, 'MASKING_IMAGE': True, 'MASKING_VALUES': [2], 'BORDER_MODE': 4, 'IMG_INTERPOLATION': 1, 'MSK_INTERPOLATION': 0, 'AUGMENT': True, 'AUGMENT_PROB': 1, 'SHUFFLE': True, 'RESAMPLE': True, 'SCALER': 'Standard', 'CONFIG_PATH': 'reports/config/temp/MotionGenerator-tests', 'TENSORBOARD_LOG_DIR': 'reports/tensorboard_logs/temp/MotionGenerator-tests', 'MODEL_PATH': 'models/temp/MotionGenerator-tests', 'HISTORY_PATH': 'reports/history/temp/MotionGenerator-tests'}


In [2]:
######################################
# for fast tests, no choice
path_to_df = '/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SAx_3D_dicomTags_phase_outcome_meta_bsa'
df = pd.read_csv(path_to_df)
df['phase'] = pd.Categorical(df.phase, 
                      categories=['ED','MS','ES','PF','MD'],
                      ordered=True)
#print(df.info())
#######################################

# load the dataframe
from ipyfilechooser import FileChooser
df_chooser = FileChooser('/mnt/ssd/data/gcn/02_imported_4D_unfiltered/')
display(df_chooser)
@interact_manual
def load_df():
    global df
    if df_chooser.selected is not None:
        path_to_df  = df_chooser.selected
        df = pd.read_csv(path_to_df)
        df['phase'] = pd.Categorical(df.phase, 
                              categories=['ED','MS','ES','PF','MD'],
                              ordered=True)
        #print(df.describe())
        print(df.info())

FileChooser(path='/mnt/ssd/data/gcn/02_imported_4D_unfiltered', filename='', title='HTML(value='', layout=Layo…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [3]:
# check internally if there are two columns for the patient ID, the original row from the Excel-Sheet had errors
print(df[['patient', 'PatientID']][df.PatientID != df.patient])
# extract the files for all phases
df_phases = df[['patient','x_path', 'phase']]
df_phases

      patient PatientID
5    04NEJQUZ  04NEJQU7
6    04NEJQUZ  04NEJQU7
7    04NEJQUZ  04NEJQU7
8    04NEJQUZ  04NEJQU7
9    04NEJQUZ  04NEJQU7
640  GZH20326  6ZH20326
641  GZH20326  6ZH20326
642  GZH20326  6ZH20326
643  GZH20326  6ZH20326
644  GZH20326  6ZH20326
890  PEXM9GEK  PEXM9GE 
891  PEXM9GEK  PEXM9GE 
892  PEXM9GEK  PEXM9GE 
893  PEXM9GEK  PEXM9GE 
894  PEXM9GEK  PEXM9GE 


,patient,x_path,phase
0,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,MS
1,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,ES
2,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,PF
3,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,MD
4,02Z1L14N,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,ED
...,...,...,...
1320,ZZQ0001Q,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,ED
1321,ZZQ0001Q,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,MS
1322,ZZQ0001Q,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,ES
1323,ZZQ0001Q,/mnt/ssd/data/gcn/02_imported_4D_unfiltered/SA...,PF


In [4]:
# zip the phase files in the following structure:
#[[patient1_phase1, patient1_phase2, patient1_phase3, patient1_phase4, patient1_phase5],[patient2_phase1...] ...]

ed = df_phases[df_phases['phase'] == 'ED']['x_path']
ms = df_phases[df_phases['phase'] == 'MS']['x_path']
es = df_phases[df_phases['phase'] == 'ES']['x_path']
pf = df_phases[df_phases['phase'] == 'PF']['x_path']
md = df_phases[df_phases['phase'] == 'MD']['x_path']

# zip the phase liss into one list of lists/tuples
inputs = list(zip(ed,ms,es,pf,md))
outputs = list(zip(ms,es,pf,md,ed))

In [5]:
# initialise a generator with these files
# logging.getLogger().setLevel(logging.INFO)
from src.data.Generators import MotionDataGenerator
config['AUGMENT'] = True
config['RESAMPLE'] = True
batch_generator = MotionDataGenerator(inputs, outputs, config=config)
val_config = config.copy()
val_config['AUGMENT'] = False
validation_generator = MotionDataGenerator(inputs, outputs , config=val_config)

2021-02-08 19:37:10,133 INFO Create DataGenerator
2021-02-08 19:37:10,138 INFO Datagenerator created with: 
 shape: [60, 192, 192]
 spacing: [2, 2, 2]
 batchsize: 1
 Scaler: Standard
 Images: 265 
 Augment: True 
 Thread workers: 1
2021-02-08 19:37:10,139 INFO Data will be augmented (shift,scale and rotate) with albumentation
2021-02-08 19:37:10,139 INFO Create DataGenerator
2021-02-08 19:37:10,144 INFO Datagenerator created with: 
 shape: [60, 192, 192]
 spacing: [2, 2, 2]
 batchsize: 1
 Scaler: Standard
 Images: 265 
 Augment: False 
 Thread workers: 1
2021-02-08 19:37:10,145 INFO No augmentation


In [16]:
# visualise one batch
@interact_manual
def select_image_in_batch(batch = (0,len(batch_generator), 1), im = (0,BATCHSIZE- 1, 1), slice_n=(1,11), save=False, filepath='data/temp/', filename='temp_x.npy'):
    
    import time
    global inputs, outputs
    t1 = time.time()
    inputs, outputs = batch_generator.__getitem__(batch)
    logging.info('Preprocessing took: {:0.3f}'.format(time.time() - t1))
    print(inputs.shape, outputs.shape)
    print('selected batch : ' + str(batch))
    
    selected_input = inputs[im]
    selected_output = outputs[im]
    
    list(map(lambda x: show_2D_or_3D(img=x[::slice_n], interpol='bilinear',dpi=100,f_size=(5,5)), selected_input))
    plt.show()
    info('-'*100)
    #list(map(lambda x: show_2D_or_3D(img=x[::slice_n], interpol='bilinear',dpi=100,f_size=(5,5)), selected_output))
    
    """
        show_2D_or_3D(x[im], y[im], interpol='bilinear',dpi=100,f_size=(5,5))
    plt.show()
    """
    plt.hist(selected_input.flatten(), bins=np.linspace(0.1,1,20))
    plt.show()
    if save:
        ensure_dir(filepath)
        np.save(os.path.join(filepath, filename), x[im])
        logging.info('saved to {}'.format(os.path.join(filepath, filename)))
    



interactive(children=(IntSlider(value=132, description='batch', max=265), IntSlider(value=0, description='im',…

In [9]:
temp = np.random.rand(200,200,200)
print(temp.shape)

(200, 200, 200)


In [15]:
%%timeit
temp.mean()

2.65 ms ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
np.mean(temp)

2.69 ms ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
np.linspace(-1,1,10)

array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])